## PREPARATION

In [ ]:
# Reference: https://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script/15622021#15622021
import sys
sys.path.append(r'S:\Grid_Ori_bigdata')

In [ ]:
import os

import numpy as np
import pandas as pd

import rasterio as rio
import xarray as xr
import rioxarray as rxr

import geopandas as gpd

from rasterio.features import shapes

from sklearn.metrics import mean_squared_error

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_style("ticks") # Ref: https://seaborn.pydata.org/tutorial/aesthetics.html

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # For creating grid spec

In [ ]:
main_dir = r"S:\Grid_Ori_bigdata"

---------------------------------------------------------------------------

## VALIDATION

#### 1. Get functions

In [ ]:
# Ref: https://gis.stackexchange.com/questions/317391/python-extract-raster-values-at-point-locations
def point_raster_join(pts_df, path):
    '''A function is to get values from raster at the points'''
    coords = [(x, y) for x, y in zip(pts_df.x, pts_df.y)]
    mx_depth = rio.open(path)
    pts_list = [x[0] for x in mx_depth.sample(coords)]
    return pts_list

In [ ]:
def get_dict(value_name, iterative_range, filename, observed_df):
    '''A function is to get values from multiple rasters at the points'''
    calibration_dict = {}
    # Looping to get data
    for i in range(len(iterative_range)):
        # Get dataframe
        path = fr"{main_dir}\\n_{n_calibration[i]}\\{filename}"
        calibration_df = observed_df.copy(deep=True)
        calibration_df[f'{value_name}'] = point_raster_join(calibration_df, path)
        calibration_df[f'{value_name}'] = calibration_df[f'{value_name}'].replace(-9999, np.nan)
        calibration_dict[f"n_{n_calibration[i]}"] = calibration_df[['level', f"{value_name}"]]
    return calibration_dict

#### 2. Get observed data

In [ ]:
# Get observed data
obs_data_df = gpd.read_file(fr"{main_dir}\2005b_Flood.shp")
# Choose geometry and level
debris_df = obs_data_df[['geometry', 'X', 'Y', 'level_']]
# Rename
debris_df.rename(columns={'X':'x', 'Y':'y', 'level_':'level'}, inplace=True)
# Copy the dataframe and call it validation dataframe
validation_df = debris_df.copy(deep=True)

In [ ]:
%%time
# Get level from model results
validation_df['mxe'] = point_raster_join(debris_df, fr"{main_dir}\test_nearest\out.mxe")

#### 3. Calculate errors

In [ ]:
# Calculate the error
validation_df_copy = validation_df.copy(deep=True)
validation_df_copy['error'] = validation_df['mxe'] - validation_df_copy['level']

In [ ]:
# Get avarge error and avarage absolute error
print(validation_df_copy['error'].mean())
print(validation_df_copy['error'].abs().mean())

#### 4. Get plot

In [ ]:
# Validate with mxe and rmse
validation_mxe_mse = mean_squared_error(validation_df.level, validation_df.mxe, squared=True)
validation_mxe_rmse = mean_squared_error(validation_df.level, validation_df.mxe, squared=False)

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(7, 7))

# Size for title and label
fontsize = 14
labelpad = 21

# Plot
sns.regplot(x='level', y='mxe', data=validation_df,
            scatter_kws={"s": 50, 'edgecolor': 'black', 'color':'deeppink', 'linewidth':.7},
            line_kws={'color':'darkred', 'linewidth':1.5}, marker='o', ci=95, ax=ax)

# Adjust x and y labels
ax.set_xlabel("Observed data - Level (m)", fontsize=fontsize, labelpad=labelpad)
ax.set_ylabel("Predicted data -\nmaximum water surface elevation (m)", rotation=-270, fontsize=fontsize, labelpad=labelpad+5)

# Set up ticks
ax.set_yticks(np.arange(2.5, 21, 2.5))

# For frame
for spine in ax.spines.values():
    spine.set_edgecolor('black')
    
# Set up ticks
for item in (ax.get_xticklabels() + ax.get_yticklabels()):  # For x, y ticks' labels
    item.set_fontsize(fontsize-3)
ax.tick_params(direction='out', length=5, pad=labelpad-17)
    

title_error = "MSE:\n\nRMSE:"
error = f"{validation_mxe_mse:.3f}\n\n{validation_mxe_rmse:.3f}"

# Error added into the text
# Ref: https://github.com/matplotlib/matplotlib/issues/253/
#      https://stackoverflow.com/questions/67366092/valueerror-alignment-not-allowed-in-string-format-specifier-sometimes-not
#      https://stackoverflow.com/questions/8234445/format-output-string-right-alignment
ax.text(
    .1, .8, # Control the text on the x axis and y axis
    title_error,
    size=fontsize-2, ha='left', color='black', transform=ax.transAxes
)
ax.text(
    .22, .8, # Control the text on the x axis and y axis
    error,
    size=fontsize-2, ha='left', color='black', transform=ax.transAxes
)

plt.savefig(fr"{main_dir}\validation_result.png", bbox_inches='tight', dpi=330)

---------------------------------------------------------------------

## CALIBRATION

#### 1. Get data

In [ ]:
# Get range of calibration
n_calibration = np.round(np.arange(0.5, 2.6, 0.1), 1)

In [ ]:
%%time
# Get dictionary of calibration results
n_dict = get_dict('mxe', n_calibration, 'out.mxe', debris_df)

#### 2. Counting NaNs

In [ ]:
# Copy dictionary
import copy
n_dict_copy_001 = copy.deepcopy(n_dict)

In [ ]:
# Counting NaNs
num_missing_values = []
for i in range(len(n_calibration)):
    nan_num = n_dict_copy_001[f"n_{n_calibration[i]}"].mxe.isna().sum()
    text = "n = {0} has {1} missing values".format(n_calibration[i], nan_num)
    num_missing_values.append(nan_num)
    print(text)

In [ ]:
# Plot missing values
fig, ax = plt.subplots(figsize=(10, 5))

ax.scatter(x=n_calibration, y=num_missing_values, c='red')
ax.plot(n_calibration, num_missing_values) # For line
ax.set_title("Missing values vs. Manning's n")
ax.set_xlabel("Manning's n")
ax.set_ylabel("Missing values")

#### 3. Get errors

In [ ]:
# Copy dictionary
n_dict_copy_002 = copy.deepcopy(n_dict)

In [ ]:
# Get no missing data dictionary
n_nomissing_dict = {}
for i in range(len(n_calibration)):
    n_nomissing_dict[f"n_{n_calibration[i]}"] = n_dict_copy_002[f"n_{n_calibration[i]}"][n_dict_copy_002['n_0.5'].mxe.notnull()]

In [ ]:
# Generate RMSE without missing values
rmse_selectedmissing_list = []
for i in range(len(n_calibration)):
    rmse = mean_squared_error(
        n_nomissing_dict[f"n_{n_calibration[i]}"][n_nomissing_dict[f"n_{n_calibration[i]}"].mxe.notnull()].level, 
        n_nomissing_dict[f"n_{n_calibration[i]}"][n_nomissing_dict[f"n_{n_calibration[i]}"].mxe.notnull()].mxe, 
        squared=False
    )
    rmse_selectedmissing_list.append(rmse)
    text = "RMSE when n = {0} is {1:.3f}".format(n_calibration[i], rmse_selectedmissing_list[i])
    print(text)

In [ ]:
# # Connected line
# fig, ax = plt.subplots(figsize=(16, 6))

# fontsize = 15
# labelpad = 21

# # Plot
# # Line
# ax.plot(n_calibration, rmse_selectedmissing_list, c='deeppink') # For line
# # Points
# ax.scatter(x=n_calibration, y=rmse_selectedmissing_list, 
#            facecolor='maroon', edgecolor='white', linewidth=1, s=70, zorder=2)

# # Set up ticks
# ax.set_xticks(np.arange(.5, 2.6, .1))
# ax.set_yticks(np.arange(.3, 1, .1))

# # Set up x limit
# ax.set_ylim(bottom=.37)

# # Adjust x and y labels
# ax.set_xlabel("Multipliers of Manning's n", fontsize=fontsize, labelpad=labelpad)
# ax.set_ylabel("RMSE\nof maximum water surface elevation (m)", rotation=-270, fontsize=fontsize, labelpad=labelpad+5)

    
# # Set up ticks
# for item in (ax.get_xticklabels() + ax.get_yticklabels()):  # For x, y ticks' labels
#     item.set_fontsize(fontsize-2)
# ax.tick_params(direction='out', length=labelpad-15, pad=labelpad-13)

# # Save
# plt.savefig(fr"{main_dir}\calibration_connectedline.png", bbox_inches='tight', dpi=330)

In [ ]:
# No connected lines
fig, ax = plt.subplots(figsize=(16, 6))

fontsize = 15
labelpad = 21

# Plot
ax.scatter(x=n_calibration, y=rmse_selectedmissing_list, 
           facecolor='deeppink', edgecolor='maroon', linewidth=1, s=70, zorder=2)

# Set up ticks
ax.set_xticks(np.arange(.5, 2.6, .1))
ax.set_yticks(np.arange(.3, 1, .1))

# Set up x limit
ax.set_ylim(bottom=.37)

# Adjust x and y labels
ax.set_xlabel("Multipliers of Manning's n", fontsize=fontsize, labelpad=labelpad)
ax.set_ylabel("RMSE\nof maximum water surface elevation (m)", rotation=-270, fontsize=fontsize, labelpad=labelpad+5)

    
# Set up ticks
for item in (ax.get_xticklabels() + ax.get_yticklabels()):  # For x, y ticks' labels
    item.set_fontsize(fontsize-2)
ax.tick_params(direction='out', length=labelpad-15, pad=labelpad-13)

# Save
plt.savefig(fr"{main_dir}\calibration_noconnectedline.png", bbox_inches='tight', dpi=330)